# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [18]:
# checking the current working directory
#print(os.getcwd())
all_files=[]
# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
        
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*s.csv'))
    for file in file_path_list:
        all_files.append(os.path.abspath(file))

# sort the files         
all_files.sort()
all_files

['/home/workspace/event_data/2018-11-01-events.csv',
 '/home/workspace/event_data/2018-11-02-events.csv',
 '/home/workspace/event_data/2018-11-03-events.csv',
 '/home/workspace/event_data/2018-11-04-events.csv',
 '/home/workspace/event_data/2018-11-05-events.csv',
 '/home/workspace/event_data/2018-11-06-events.csv',
 '/home/workspace/event_data/2018-11-07-events.csv',
 '/home/workspace/event_data/2018-11-08-events.csv',
 '/home/workspace/event_data/2018-11-09-events.csv',
 '/home/workspace/event_data/2018-11-10-events.csv',
 '/home/workspace/event_data/2018-11-11-events.csv',
 '/home/workspace/event_data/2018-11-12-events.csv',
 '/home/workspace/event_data/2018-11-13-events.csv',
 '/home/workspace/event_data/2018-11-14-events.csv',
 '/home/workspace/event_data/2018-11-15-events.csv',
 '/home/workspace/event_data/2018-11-16-events.csv',
 '/home/workspace/event_data/2018-11-17-events.csv',
 '/home/workspace/event_data/2018-11-18-events.csv',
 '/home/workspace/event_data/2018-11-19-events

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [19]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []

# for every filepath in the file path list 
for f in all_files:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        #creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
         # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
i=0
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            
            continue
            
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        
        

In [4]:
len(full_data_rows_list)

8056

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [20]:
data= pd.read_csv('event_datafile_new.csv')
#data[data.userId == 88]
data.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Des'ree,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",139,You Gotta Be,8
1,Mr Oizo,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",139,Flat 55,8
2,Tamba Trio,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",139,Quem Quiser Encontrar O Amor,8
3,The Mars Volta,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",139,Eriatarka,8
4,Infected Mushroom,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",139,Becoming Insane,8


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [2]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [3]:
#Create a Keyspace 

try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = {'class':'SimpleStrategy','replication_factor':1}""")
    
except Exception as e:
    print(e)

#### Set Keyspace

In [4]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [5]:
##Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

try:
    drop_query_1 ="DROP TABLE IF EXISTS artist_and_song_by_sessionid"
    session.execute(drop_query_1)
    
except Exception as e:
    print(e)
    
try:
    create_query_1 ="CREATE TABLE IF NOT EXISTS artist_and_song_by_sessionid (session_id int,item_insession int,\
                artist text,song_title text,song_length double, PRIMARY KEY (session_id,item_insession)) " 
 
    session.execute(create_query_1)
except Exception as e:
    print(e)

                    

In [6]:
# inserting data into the table from the event_datafile file
try:
    file = 'event_datafile_new.csv'

    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:

            query = "INSERT INTO artist_and_song_by_sessionid \
                    (session_id,item_insession,artist,song_title,song_length)"
            query = query + "VALUES (%s,%s,%s,%s,%s)"
        
            session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))
        
except Exception as e:
    print(e)

The table for this query uses the session_id as the partitioning key to allocate same session_id to thesame node and the item_insession is use for sorting the record in each node. With this, reading out the artist, song_title listened to, and song_length during a session only requires identifying the node the value "338" is partition and looking up the sorted column (item_insession) for the corresponding value "4". 

#### Do a SELECT to verify that the data have been inserted into each table

In [7]:
##verifying the data was entered into the table
try:
    select_query_1 ="SELECT artist,song_title,song_length FROM artist_and_song_by_sessionid \
    WHERE session_id= 338 AND item_insession= 4" 
 
    results = session.execute(select_query_1)
    for row in results:
        print(row.artist,'|',row.song_title,'|',row.song_length)
except Exception as e:
    print(e)

Faithless | Music Matters (Mark Knight Dub) | 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [8]:
##Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


try:
    drop_query_2 ="DROP TABLE IF EXISTS artist_info_by_userid"
    session.execute(drop_query_2)
    
except Exception as e:
    print(e)
    
try:
    create_query_2 ="CREATE TABLE IF NOT EXISTS artist_info_by_userid (user_id int,session_id int,item_insession int,\
    artist text,song_title text,user_first_name text,user_last_name text,\
    PRIMARY KEY ((user_id,session_id),item_insession)) " 
 
    session.execute(create_query_2)
except Exception as e:
    print(e)


                    

In [9]:
# inserting data into the table from the event_datafile file

try:
    file = 'event_datafile_new.csv'

    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
    
            query = "INSERT INTO artist_info_by_userid (user_id,session_id,item_insession,\
                        artist,song_title,user_first_name,user_last_name) "
            query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
            
            
            session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))
        
except Exception as e:
    print(e)

The table created for this query, uses the user_id and session_id as the composite partition key to partition the records. A better and more balance distribution is obtain with this composite key, than using a single column like the user_id, since some users visits the app more frequent than others. Adding the session_id to the partition key will force thesame user_id with same session_id to be on thesame node thereby reducing the initial skewness (using only the user_id). Obtaining the query result requires hashing the partition key values "10" and "182" and identifying the node it is partition. The item_insession column is also used to sort the records in each node, making it efficient for selection. 

In [10]:
##verifying the data was entered into the table

try:
    select_query_2 = "SELECT artist,song_title,user_first_name,user_last_name FROM artist_info_by_userid \
    WHERE user_id= 10 AND session_id= 182" 
 
    results = session.execute(select_query_2)
    for row in results:
        print(row.artist,'|',row.song_title,'|',row.user_first_name,'|',row.user_last_name)
except Exception as e:
    print(e)

Down To The Bone | Keep On Keepin' On | Sylvie | Cruz
Three Drives | Greece 2000 | Sylvie | Cruz
Sebastien Tellier | Kilometer | Sylvie | Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie | Cruz


In [11]:
##Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



try:
    
    drop_query_3 ="DROP TABLE IF EXISTS user_by_song_heard"
    session.execute(drop_query_3)
    
except Exception as e:
    print(e)
    
try:
    create_query_3 ="CREATE TABLE IF NOT EXISTS user_by_song_heard (song_title text,user_id int,\
                        user_first_name text,user_last_name text,PRIMARY KEY (song_title,user_id))" 
 
    session.execute(create_query_3)
except Exception as e:
    print(e)

                    

In [12]:
# inserting data into the table from the event_datafile file


try:
    file = 'event_datafile_new.csv'

    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
    
            query = "INSERT INTO user_by_song_heard (song_title,user_id,user_first_name,user_last_name)"
            query = query + "VALUES (%s,%s,%s,%s)"
            
            
            session.execute(query, (line[9],int(line[10]),line[1],line[4]))
        
except Exception as e:
    print(e)

The table created for this query uses the song_title and user_id as the primary key, which is assumed to be unique. This will not be unique if a user can repeat a song, another suggestion will be to add the session_id to the primary key, meaning a user can't repeat thesame song in thesame session. For a case where a user can't repeat thesame song, the partitioning key being the song_title is used to partition the records, and obtaining the result of users who listened to a particular song will only invole records from a single node.

In [13]:
#verifying the data was entered into the table

try:
    select_query_3 ="SELECT user_first_name,user_last_name FROM user_by_song_heard \
    WHERE song_title = 'All Hands Against His Own' " 
 
    results = session.execute(select_query_3)
    for row in results:
        print(row.user_first_name,'|',row.user_last_name)
except Exception as e:
    print(e)

Jacqueline | Lynch
Tegan | Levine
Sara | Johnson


### Drop the tables before closing out the sessions

In [14]:
try:
    drop_query_1 ="DROP TABLE IF EXISTS artist_and_song_by_sessionid"
    session.execute(drop_query_1)
    
    drop_query_2 ="DROP TABLE IF EXISTS artist_info_by_userid"
    session.execute(drop_query_2)
    
    drop_query_3 ="DROP TABLE IF EXISTS user_by_song_heard"
    session.execute(drop_query_3)
    
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()